In [1]:
import warnings 
warnings.filterwarnings('ignore')
import time 
import datetime
import pandas as pd 
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
from scipy import stats
from scipy.stats import norm, skew
from scipy.special import boxcox1p
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC, Ridge, LinearRegression
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor, AdaBoostRegressor, GradientBoostingClassifier
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor, ExtraTreesRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import RobustScaler, StandardScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
import lightgbm as lgb
import xgboost as xgb
from sklearn.svm import SVR
from mlxtend.regressor import StackingRegressor, StackingCVRegressor
from sklearn.metrics import make_scorer
from sklearn import model_selection 
from collections import defaultdict
from sklearn.dummy import DummyRegressor
from sklearn.ensemble.partial_dependence import plot_partial_dependence
from catboost import CatBoostRegressor

import statsmodels.api as sm

from scipy.stats import spearmanr

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 12, 4

In [2]:
def mape_func(labels, preds):
    return np.mean(np.abs((preds - labels)/(labels))) * 100

mape_score = make_scorer(mape_func)#, greater_is_better=False)

# Data gathering 

In [3]:
data      = pd.read_csv('/Users/mohsenkiskani/.kaggle/competitions/ubaar-competition/train.csv')
test_data = pd.read_csv('/Users/mohsenkiskani/.kaggle/competitions/ubaar-competition/test.csv')

# Remove NANs
data      = data.dropna(axis = 0)

# Remove outliers
data.drop([28098])
THRESHOLD = 4.5e7
Aa = data[data.price > THRESHOLD]
data = data.drop(Aa.index.tolist())

specific_cols = ['distanceKM', 'taxiDurationMin', 'weight']
removed_indices = []
for col in specific_cols:
    df = data['price']/data[col]
    A = df[~df.isin([np.nan, np.inf, -np.inf])]
    B = (A - np.mean(A)) / np.std(A)
    V = B[B > 5]
    removed_indices.extend(V.index.tolist())
data = data.drop(set(removed_indices))

# Fill test NANs
test_data.loc[12577, 'distanceKM']      = 52
test_data.loc[12577, 'taxiDurationMin'] = 50
test_data.loc[13853, 'distanceKM']      = 500
test_data.loc[13853, 'taxiDurationMin'] = 380

all_data = pd.concat((data, test_data)) 
all_data['source']           = all_data['sourceLatitude']*all_data['sourceLongitude']
all_data['destination']      = all_data['destinationLatitude']*all_data['destinationLongitude']

#all_data['dist2']      = all_data['distanceKM']*all_data['distanceKM']
#all_data['dist3']      = all_data['dist2']*all_data['distanceKM']
#all_data['taxi2']      = all_data['taxiDurationMin']*all_data['taxiDurationMin']
#all_data['taxi3']      = all_data['taxi2']*all_data['taxiDurationMin']
#all_data['weight2']      = all_data['weight']*all_data['weight']
#all_data['weight3']      = all_data['weight2']*all_data['weight']
#all_data['source2']      = all_data['source']*all_data['source']
#all_data['destination2']      = all_data['destination']*all_data['destination']
#all_data['destinationlat2']      = all_data['destinationLatitude']*all_data['destinationLatitude']
#all_data['destinationlon2']      = all_data['destinationLongitude']*all_data['destinationLongitude']

ntrain = data.shape[0]
ntest  = test_data.shape[0]

categorical_vars = ['date', 'SourceState', 'destinationState', 'vehicleType', 'vehicleOption']

dummies_data = pd.get_dummies(all_data[categorical_vars])
all_data[dummies_data.columns] = dummies_data[dummies_data.columns]
all_data.drop(categorical_vars, axis=1, inplace=True)

train    = all_data[:ntrain]
test     = all_data[ntrain:]

feat_names = all_data.columns.tolist()
feat_names.remove('ID')
feat_names.remove('price')

X = train.drop(['ID','price'],axis=1)
y = train.price

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
X_train.shape

(39645, 84)

In [4]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=3):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                #instance.fit(X[train_index], y[train_index])
                instance.fit(X.iloc[train_index], y.iloc[train_index])
                y_pred = instance.predict(X.iloc[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [5]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)

In [6]:
res = defaultdict(dict)

def benchmark(est, name=None, cv=False):
    if not name:
        name = est.__class__.__name__
    print("Started benchmarking    " + name + "        at time: ", datetime.datetime.now())
    if not cv:
        t0 = time.time()
        est.fit(X_train, y_train)
        res[name]['train_time'] = (time.time() - t0)/60
        t0 = time.time()
        pred = est.predict(X_val)
        res[name]['test_time'] = (time.time() - t0)/60
        res[name]['MAPE'] = mape_func(y_val, pred)
    else:
        t0 = time.time()
        res[name]['cv_score'] = np.mean(model_selection.cross_val_score(est, X, y, scoring=mape_score, cv = 3))
        res[name]['cv_time'] = (time.time() - t0)/60
    print("Done benchmarking       " + name + "        at time: ", datetime.datetime.now())
    return est

In [7]:
randomForest = RandomForestRegressor(n_estimators=800, max_features=17, random_state=5, bootstrap=False, n_jobs=10)
extraTree    = ExtraTreesRegressor(n_estimators=3000, max_features=23, random_state=5, bootstrap=False, n_jobs=4)
adaBoost     = AdaBoostRegressor(n_estimators=600, learning_rate=0.01, loss='exponential', random_state=5)
decTree      = DecisionTreeRegressor( splitter='best', max_depth=16, min_samples_split=20, 
                             min_samples_leaf=10, min_weight_fraction_leaf=0.0, max_features=None, 
                             random_state=5, max_leaf_nodes=None, min_impurity_decrease=0.1, 
                             min_impurity_split=None, presort=False)
bagging      = BaggingRegressor(n_estimators=600, max_samples=1.0, max_features=0.9, random_state=5, verbose=1)
ridge        = Ridge(alpha=0.0001, normalize=True)
knn          = KNeighborsRegressor(2)
lasso        = Lasso(fit_intercept = True, random_state=5)
enet         = make_pipeline(RobustScaler(), ElasticNet(alpha=0.8, l1_ratio=.9, random_state=5))
xgboosting   = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                        learning_rate=0.1, max_depth=12, 
                        min_child_weight=1.7817, n_estimators=800,
                        reg_alpha=0.9640, reg_lambda=0.8571,
                        subsample=1, silent=1,
                        random_state =5 , nthread = -1)
gboosting    = GradientBoostingRegressor(n_estimators=15000, learning_rate=0.01,
                                  max_depth=10, max_features='sqrt',
                                  min_samples_leaf=15, min_samples_split=10, 
                                  loss='huber', random_state = 42)
lightgbm     = lgb.LGBMRegressor(objective='regression',num_leaves=25, save_binary = True,  
                          learning_rate=0.01, n_estimators=60000,
                          max_bin = 150, bagging_fraction = 0.95,
                          bagging_freq = 4, feature_fraction = 0.8,
                          feature_fraction_seed=50, bagging_seed=20,
                          min_data_in_leaf = 11, min_sum_hessian_in_leaf = 11)

In [9]:
models = [gboosting0]

for model in models:
    benchmark(model, cv=False)

Started benchmarking    GradientBoostingRegressor        at time:  2018-06-30 00:13:36.109671
Done benchmarking       GradientBoostingRegressor        at time:  2018-06-30 00:16:03.506403


In [10]:
res_df = pd.DataFrame(data=res).T
res_df.sort_values('MAPE')

,MAPE,test_time,train_time
GradientBoostingRegressor,17.20314,0.009051,2.447549


In [ ]:
feature_importances = pd.Series(xgboosting.feature_importances_, X_train.columns.values)
feature_importances.to_pickle('dataFrames/xgboost_feature_importances') 
feature_importances = feature_importances.sort_values(ascending=False)
#feature_importances = plt.plot()
plt.figure(num=None, figsize=(20, 10), dpi=80, facecolor='w', edgecolor='k')

feature_importances.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')

In [11]:
avg_lgb_rfst = AveragingModels(models = (lightgbm0, randomForest, gboosting0))
avg_lgb_rfst.fit(X_train, y_train)

AveragingModels(models=(LGBMRegressor(bagging_fraction=0.95, bagging_freq=4, bagging_seed=20,
       boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
       feature_fraction=0.8, feature_fraction_seed=50, learning_rate=0.01,
       max_bin=150, max_depth=-1, min_child_samples=20,
       min_child_weig...          presort='auto', random_state=42, subsample=1.0, verbose=0,
             warm_start=False)))

In [ ]:
pred = avg_lgb_rfst.predict(X_val)
mape_func(y_val, pred)

In [ ]:
stack_gen = StackingRegressor(regressors=( lightgbm0, randomForest), 
                               meta_regressor=ridge )
name = stack_gen.__class__.__name__
t0 = time.time()
stack_gen.fit(X_train.as_matrix(), y_train.as_matrix())
res[name]['train_time'] = (time.time() - t0)/60
t0 = time.time()
pred = stack_gen.predict(X_val.as_matrix())
res[name]['test_time'] = (time.time() - t0)/60
res[name]['MAPE'] = mape_func(y_val, pred)

Stackgen results in higher scores. LGBM gives 17.42 and Gboost gives 17.2 yet Stackgen on these two base models with ridge as the meta-model gives 17.808!

In [ ]:
name = stack_gen.__class__.__name__
res[name]['train_time'] =  50
t0 = time.time()
pred = stack_gen.predict(X_val.as_matrix())
res[name]['test_time'] = (time.time() - t0)/60
res[name]['MAPE'] = mape_func(y_val, pred)

res_df = pd.DataFrame(data=res).T
res_df.to_pickle('dataFrames/benchmarking_results_stack_gen.pkl')
res_df.sort_values('MAPE')

In [ ]:
#stack_gen.__class__.__name__
#stack_gen.fit(X_train.as_matrix(), y_train.as_matrix())
#benchmark(stack_gen, cv=False)

In [ ]:
#stacked_averaged_models  = StackingAveragedModels(base_models = (xgboosting, randomForest), meta_model = ridge)
stacked_averaged_models  = StackingAveragedModels(base_models = (lightgbm, extraTree), meta_model = ridge)
stacking_regressor       = StackingRegressor(regressors=[xgboosting, randomForest], meta_regressor = ridge)



In [ ]:
res_df = pd.read_pickle('dataFrames/benchmarking_results.plk')
res_df.sort_values('MAPE')

In [ ]:
res_df = pd.DataFrame(data=res).T
res_df.to_pickle('dataFrames/benchmarking_results2.plk')
res_df.sort_values('MAPE')

In [ ]:
models = [lightgbm, xgboosting, enet, lasso, ridge]

In [ ]:
avg_lgbm_xgb  = StackingAveragedModels(base_models = (lightgbm, xgboosting), meta_model = ridge)
benchmark(avg_lgbm_xgb, name="Avg_LGBM_XGB", cv=False)

In [ ]:
stacking_regressor       = StackingRegressor(regressors=[lightgbm, xgboosting], meta_regressor = ridge)
benchmark(avg_lgbm_xgb, name="Stacking_LGBM_XGB", cv=False)

In [ ]:
avg_lgbm_gboost  = StackingAveragedModels(base_models = (lightgbm, gboosting), meta_model = ridge)
benchmark(avg_lgbm_gboost, name="Avg_LGBM_Gboost", cv=False)

In [ ]:
res_df = pd.DataFrame(data=res).T
res_df.to_pickle('dataFrames/benchmarking_results2.pkl')
res_df.sort_values('MAPE')

In [ ]:
res_df = pd.read_pickle('dataFrames/benchmarking_results.plk')
res_df.sort_values('MAPE')

In [ ]:
# Computationally intensive 
#KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
#KRR.fit(X_train, y_train)
#get_score(KRR,X_val,y_val)

# score = 33.473933712491295

In [ ]:
# Very computationally expensive
#BAYSR = BayesianRidge(n_iter=300, tol=0.001, alpha_1=1e-06, alpha_2=1e-06, lambda_1=1e-06, lambda_2=1e-06, 
#                      compute_score=True, fit_intercept=True, normalize=True)
#BAYSR.fit(X_train, y_train)
#get_score(BAYSR,X_val,y_val)

### Testing stackingRegressor

Look at the following link: https://www.kaggle.com/laurenstc/top-2-of-leaderboard-advanced-fe

### Testing another stacking method

Look at the following link:
https://www.kaggle.com/serigne/stacked-regressions-top-4-on-leaderboard

In [ ]:
def deviance_plot(est, X_test, y_test, ax=None, label='', train_color='#2c7bb6', 
                  test_color='#d7191c', alpha=1.0, ylim=(0, 10)):
    """Deviance plot for ``est``, use ``X_val`` and ``y_val`` for test error. """
    n_estimators = len(est.estimators_)
    test_dev = np.empty(n_estimators)

    for i, pred in enumerate(est.staged_predict(X_test)):
        test_dev[i] = est.loss_(y_test, pred)

    if ax is None:
        fig = plt.figure(figsize=FIGSIZE)
        ax = plt.gca()
        
    ax.plot(np.arange(n_estimators) + 1, test_dev, color=test_color, label='Test %s' % label, 
             linewidth=2, alpha=alpha)
    ax.plot(np.arange(n_estimators) + 1, est.train_score_, color=train_color, 
             label='Train %s' % label, linewidth=2, alpha=alpha)
    ax.set_ylabel('Error')
    ax.set_xlabel('n_estimators')
    ax.legend(loc='upper right')
    #ax.set_ylim(ylim)
    return test_dev, ax

In [ ]:
def fmt_params(params):
    return ", ".join("{0}={1}".format(key, val) for key, val in params.items())

#annotation_kw = {'xycoords': 'data', 'textcoords': 'data',
#                 'arrowprops': {'arrowstyle': '->', 'connectionstyle': 'arc'}}



FIGSIZE = (20, 10)
fig = plt.figure(figsize=FIGSIZE)
ax = plt.gca()
for params, (test_color, train_color) in [({}, ('#d7191c', '#2c7bb6')),
                                          ({'min_samples_leaf': 3}, ('#fdae61', '#abd9e9')),
                                          ({'min_samples_leaf': 15}, ('#f12e61', '#12d9e9'))]:
    est = GradientBoostingRegressor(n_estimators=200, max_depth=1, 
                                    learning_rate=1.0)
#    est = GradientBoostingRegressor(n_estimators=1500, learning_rate=1.0,
#                                     max_depth=10, max_features='sqrt',
#                                     min_samples_split=10, loss='huber')
    est.set_params(**params)
    est.fit(X_train, y_train)
    test_dev, ax = deviance_plot(est, X_val, y_val, ax=ax, label=fmt_params(params),
                                 train_color=train_color, test_color=test_color)
    
#ax.annotate('Higher bias', xy=(100, est.train_score_[89]), xytext=(100, 3), **annotation_kw)
#ax.annotate('Lower variance', xy=(100, test_dev[89]), xytext=(100, 3.5), **annotation_kw)
plt.legend(loc='upper right')

In [ ]:
test_dev, ax = deviance_plot(est, X_val, y_val)
ax.legend(loc='upper right')

In [ ]:
X_df = pd.DataFrame(data=X_train, columns=feat_names)
X_df['price_Val'] = y_train
_ = X_df.hist(column=['source', 'destination', 'distanceKM', 'weight', 
                      'destinationLatitude', 'destinationLongitude', 
                      'sourceLatitude', 'sourceLongitude', 'taxiDurationMin'
                     ], figsize=FIGSIZE)

In [ ]:
fx_imp = pd.Series(est.feature_importances_, index=names)
fx_imp /= fx_imp.max()  # normalize
fx_imp.sort_values()
fx_imp.plot(kind='barh', figsize=FIGSIZE)

In [ ]:
features = ['destination', 'source', 'weight', 'distanceKM', 'taxiDurationMin',
            ('sourceLatitude', 'sourceLongitude')]
fig, axs = plot_partial_dependence(est, X_train, features, feature_names=names, 
                                   n_cols=2, figsize=FIGSIZE)

In [ ]:
cb_model = CatBoostRegressor(iterations=5000,
                             learning_rate=0.1,
                             depth=15,
                             eval_metric='MAPE',
                             random_seed = 7,
                             bagging_temperature = 1.4,
                             od_type='Iter',
                             metric_period = 100,
                             
                             use_best_model=True,
                             od_wait=20)

cb_model.fit(X_train, y_train, eval_set=(X_val, y_val),)
pred = cb_model.predict(X_val)
score = mape_func(y_val, pred)
score

In [ ]:
feature_importances = pd.Series(cb_model.feature_importances_, X_train.columns.values)
feature_importances.to_pickle('dataFrames/catboost_feature_importances') 

In [ ]:
feature_importances = feature_importances.sort_values(ascending=False)


#feature_importances = plt.plot()
plt.figure(num=None, figsize=(20, 10), dpi=80, facecolor='w', edgecolor='k')

feature_importances.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')

In [ ]:
%store feature_importances 

In [ ]:
benchmark(cb_model, cv= False)

In [ ]:
res_df = pd.DataFrame(data=res).T
res_df.sort_values('MAPE')

In [ ]:

cb_model.fit(X, y,
             eval_set=(X_val, y_val),
             use_best_model=True,
             verbose=True)

In [ ]:
cat_model = CatBoostRegressor(rsm=0.8, depth=5, learning_rate=0.037, eval_metric='MAPE')

In [ ]:
models = [cb_model]

for model in models:
    benchmark(model, cv=False)

In [ ]:
res_df = pd.DataFrame(data=res).T
#res_df.to_pickle('dataFrames/benchmarking_results2.pkl')
res_df.sort_values('MAPE')

In [ ]:
for pred in gbst1.staged_predict(X_train):
    plt.plot(X_train.distanceKM, pred)

In [ ]:
n_esimators = 200
test_score = np.empty(len(gbst1.estimators_))
for i, pred in enumerate(gbst1.staged_predict(X_val)):
    test_score[i] = gbst1.loss_(y_val, pred)
plt.plot(np.arange(n_esimators)+1, test_score, label='Val')
plt.plot(np.arange(n_esimators)+1, gbst1.train_score_ , label='Train')

In [ ]:
gbst2 = GradientBoostingRegressor(n_estimators=200, learning_rate=0.1,
                                  max_depth=15, max_features='sqrt',
                                  min_samples_leaf=15, min_samples_split=10, 
                                  loss='huber', random_state = 5)

In [ ]:
gbst2.fit(X_train, y_train)

In [ ]:
for pred in gbst2.staged_predict(X_train):
    plt.plot(X_train.distanceKM, pred)

In [ ]:
n_esimators = 200
test_score = np.empty(len(gbst2.estimators_))
for i, pred in enumerate(gbst2.staged_predict(X_val)):
    test_score[i] = gbst2.loss_(y_val, pred)
plt.plot(np.arange(n_esimators)+1, test_score, label='Val')
plt.plot(np.arange(n_esimators)+1, gbst2.train_score_ , label='Train')
plt.show()

In [ ]:
feature_importances = pd.Series(gbst1.feature_importances_, X_train.columns.values)
feature_importances = feature_importances.sort_values(ascending=False)


#feature_importances = plt.plot()
plt.figure(num=None, figsize=(20, 10), dpi=80, facecolor='w', edgecolor='k')

feature_importances.plot(kind='bar', title='Feature Importances')
plt.ylabel('Feature Importance Score')


In [ ]:
dtrain = xgb.DMatrix(X_train.as_matrix(), label=y_train.as_matrix())
dval   = xgb.DMatrix(X_val.as_matrix(), label=y_val.as_matrix())

In [ ]:
param = {'colsample_bytree': 0.4603, 'gamma' : 0.0468, 'learning_rate' : 0.01, 'max_depth': 12, 
         'min_child_weight' : 1.7817, 'n_estimators': 1086, 'reg_alpha' : 0.9640,'reg_lambda' : 0.8571,
         'subsample' : 1, 'silent': 1, 'random_state' :5 , 'nthread' : -1}

watchlist = [(dval, 'eval'), (dtrain, 'train')]
num_round = 108

In [ ]:
def MAPQEobj(preds, dtrain):
    labels = dtrain.get_label()
    grad = (2/(np.square(labels)))*(preds - labels)
    hess = (2/(np.square(preds)))
    grad = 2*(preds-labels)
    hess = 2*np.ones(len(grad))
    return grad , hess

def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'error', np.sum(np.square(preds-labels)) 

bst  = xgb.train(param, dtrain , 1200, watchlist, obj = MAPQEobj, feval= mohsen_evalerror)

In [ ]:
c_mohsen = 1 

def mohsen_obj(preds, dtrain): 
    labels       = dtrain.get_label()
    numb         = dtrain.num_row() 
    percent_diff = (preds-labels)/labels
    grad         = percent_diff / ((numb)*(labels)*(np.sqrt(percent_diff**2 + c_mohsen)))
    hess         = c_mohsen / ((numb)*(labels**2)*((percent_diff**2 + c_mohsen)**1.5))
    return grad , hess

def mohsen_evalerror(preds, dtrain):
    labels       = dtrain.get_label()
    percent_diff = (preds-labels)/labels
    mohsen_error = np.mean(np.sqrt(percent_diff**2+c_mohsen))
    return 'error', mohsen_error

bst  = xgb.train(param, dtrain , 20, watchlist, obj = mohsen_obj, feval= mohsen_evalerror)

#fftf = xgb.cv(param, dtrain)

#benchmark(xgboosting2, name="XGB_Test", cv=False)

In [ ]:
def logregobj(preds, dtrain):
    labels = dtrain.get_label()
    preds = 1.0 / (1.0 + np.exp(-preds))
    grad = preds - labels
    hess = preds * (1.0 - preds)
    return grad, hess

def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    # return a pair metric_name, result
    # since preds are margin(before logistic transformation, cutoff at 0)
    return 'error', float(sum(labels != (preds > 0.0))) / len(labels)

asbdjfn  = xgb.train(param, dtrain , 20, watchlist,   obj = mohsen_obj, feval= evalerror)

In [ ]:
#gbm0 = GradientBoostingRegressor(random_state=5, learning_rate=0.1,  min_samples_leaf = 1,  
#                                 max_features = 'sqrt', n_estimators = 170, loss='huber')
#param_test1 = {'min_samples_split':range(200,1001,200),
#              'max_depth':range(40,70,10)}
#gsearch1 = GridSearchCV(estimator = gbm0, param_grid = param_test1, cv = 2)
#modelfit(gbm0, X_train, y_train , X_val, y_val, printFeatureImportance=False)

In [ ]:
lasso = Lasso()

clf_lasso = GridSearchCV(lasso, {'alpha': [1e-3,1e-2,1e-1,1,1e1,1e2,1e3]}, verbose=1)
clf_lasso.fit(X_train,y_train)
print(clf_lasso.best_params_)
get_score(clf_lasso,X_val,y_val)
# {'alpha': 100.0}
# 40.78453998923231

# Initial Modelling

In [ ]:
#statsmodel_regression = sm.OLS(y_train, X_train).fit()
#statsmodel_regression.summary()
# Never try OLS again -----> Super computationally expensive

In [ ]:
start_time = time.time()
xgb1 = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                        learning_rate=0.01, max_depth=12, 
                        min_child_weight=1.7817, n_estimators=18000,
                        reg_alpha=0.9640, reg_lambda=0.8571,
                        subsample=1, silent=1,
                        random_state =5 , nthread = -1)

xgb1.fit(X_train, y_train, early_stopping_rounds=5, 
         eval_set=[(X_val[X_train.columns.tolist()],  y_val)], verbose=True)

r = xgb1.predict(X_val[X_train.columns.tolist()])
score = mean_absolute_precision_error(r, y_val)
print( '%.2f' % float((time.time() - start_time)/60 ) +" mins, score= ", '%.2f' % score)

In [ ]:
LGBF = lgb.LGBMRegressor(objective='regression',num_leaves=25, save_binary = True,  
                          learning_rate=0.005, n_estimators=120000,
                          max_bin = 150, bagging_fraction = 0.95,
                          bagging_freq = 4, feature_fraction = 0.8,
                          feature_fraction_seed=50, bagging_seed=20,
                          min_data_in_leaf = 11, min_sum_hessian_in_leaf = 11)

GBSTF = GradientBoostingRegressor(n_estimators=15000, learning_rate=0.01,
                                  max_depth=10, max_features='sqrt',
                                  min_samples_leaf=15, min_samples_split=10, 
                                  loss='huber', random_state = 42)

XGBF = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                        learning_rate=0.01, max_depth=12, 
                        min_child_weight=1.7817, n_estimators=8000,
                        reg_alpha=0.9640, reg_lambda=0.8571,
                        subsample=1, silent=1,
                        random_state =5 , nthread = -1)

BAGF  = BaggingRegressor(n_estimators=600, max_samples=1.0, max_features=0.9, random_state=5, verbose=1)

DECF  = DecisionTreeRegressor(max_depth=15)

RFSTF = RandomForestRegressor(n_estimators=100, criterion='mae', bootstrap=False,
                              max_depth=4, max_features=5,#'sqrt',
                              min_samples_leaf=5, min_samples_split =3, random_state = 42)

KNNF  = KNeighborsClassifier(2)
LASSF = Lasso(fit_intercept = True)
ABSTF = AdaBoostRegressor(n_estimators=1, learning_rate=0.01, loss='linear', random_state=5)

SVRF = SVR()
RDGF = Ridge()
ENTF = make_pipeline(RobustScaler(), ElasticNet(alpha=0.8, l1_ratio=.9, random_state=3))

models = { "Gboost": GBSTF, "xgb": XGBF, "bagging": BAGF, "lgbm": LGBF, "dec_tree": DECF, "Random_forest": RFSTF,
          "knn": KNNF, "elasticNet": ENTF, "ridge": RDGF, "lasso": LASSF, "AdaBoost": ABSTF, "SVR": SVRF}

#models = { "Gboost": GBSTF, "xgb": XGBF, "bagging": BAGF, "lgbm": LGBF}
 
#models = {"lgb0": lgb0, "xgb0": xgb0, "gbstf0": gbstf0}

#models ={ "bag0": bag0}

#models = {"RFSTF": RFSTF}

for model_name in models:
    model = models[model_name]
    start_time = time.time()
    print(datetime.datetime.now())
    model.fit(X_train, y_train)
    train_cols = X_train.columns.tolist()
    X_val['y_' + model_name] = model.predict(X_val[train_cols])
    score = mape_func(y_val, X_val['y_' + model_name])
    print(model_name, '%.2f' % float((time.time() - start_time)/60 ) +" mins, score= ", '%.2f' % score)

#X_val.to_pickle('dataFrames/One_Hot_X_val.pkl')

# Correct outputs are as followed

#Gboost        23.55 mins, score=  16.76
#xgb           25.40 mins, score=  17.36
#bagging       7.83  mins, score=  18.61
#lgbm          12.65 mins, score=  17.28
#dec_tree      4.95  mins, score=  21.45
#Random_forest 5.56  mins, score=  39.04
#knn           0.02  mins, score=  24.62
#elasticNet    0.01  mins, score=  34.81
#ridge         0.00  mins, score=  36.68
#lasso         0.10  mins, score=  36.63
#AdaBoost      0.00  mins, score=  38.60
#SVR           2.94  mins, score=  72.61
# KernelRidge and LinearRegression() take more than 1 hour to run! Don't Run them. 

In [ ]:
score = mape_func(y_val, X_val['y_' + model_name])
print(model_name, '%.2f' % float((time.time() - start_time)/60 ) +" mins, score= ", '%.2f' % score)

In [ ]:
train_cols = X_train.columns.tolist()
X_val['y_avg_boost'] = gbstf0.predict(X_val[train_cols])
score = mean_absolute_precision_error(X_val['y_avg_boost'], y_val)
print(model_name, '%.2f' % float((time.time() - start_time)/60 ) +" mins, score= ", '%.2f' % score)

In [ ]:
X_val['y_gbstf0'] =  gbstf0.predict(X_val[train_cols])

In [ ]:
X_val.head()

In [ ]:
X_e = (X_val['y_lgb0'] + X_val['y_xgb0'])/2

In [ ]:
X_h = ( X_val['y_xgb0'] * X_val['y_gbstf0'])**(1/2)

In [ ]:
score = mean_absolute_precision_error(X_h, y_val)
score

# Plotting predictions

In [ ]:
X_val = pd.read_pickle('dataFrames/One_Hot_X_val_new.pkl')
X_val.head()

In [ ]:
pred_cols = ['y_Gboost', 'y_xgb', 'y_bagging', 'y_lgbm', 'y_dec_tree', 'y_Random_forest', 'y_knn',
             'y_elasticNet', 'y_ridge', 'y_lasso', 'y_AdaBoost', 'y_SVR']

best_cols = ['y_Gboost', 'y_xgb', 'y_bagging', 'y_lgbm']
best_pred = X_val[list(best_cols)]
best_pred.head()

In [ ]:
for col in pred_cols:
    plt.scatter(X_val[col],y_val)
    plt.xlabel('Price')
    plt.ylabel(col)
    plt.title('Prediction vs Price')
    plt.show()

# Stacking 

In [ ]:
regressors = [lgb0, xgb0]
stregr     = StackingRegressor(regressors=regressors, meta_regressor=gbstf0)
start_time = time.time()
stregr.fit(X_train, y_train)
y_pred = stregr.predict(X_val[train_cols])
score  = mean_absolute_precision_error(y_pred, y_val)
print("stackingRegressor model", '%.2f' % float((time.time() - start_time)/60 ) +" mins, score= ", '%.2f' % score)
# stackingRegressor model 43.59 mins, score=  18.15

In [ ]:
class StackingAveragedModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, base_models, meta_model, n_folds=3):
        self.base_models = base_models
        self.meta_model = meta_model
        self.n_folds = n_folds
   
    # We again fit the data on clones of the original models
    def fit(self, X, y):
        self.base_models_ = [list() for x in self.base_models]
        self.meta_model_ = clone(self.meta_model)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        # Train cloned base models then create out-of-fold predictions
        # that are needed to train the cloned meta-model
        out_of_fold_predictions = np.zeros((X.shape[0], len(self.base_models)))
        for i, model in enumerate(self.base_models):
            for train_index, holdout_index in kfold.split(X, y):
                instance = clone(model)
                self.base_models_[i].append(instance)
                #instance.fit(X[train_index], y[train_index])
                instance.fit(X.iloc[train_index], y.iloc[train_index])
                y_pred = instance.predict(X.iloc[holdout_index])
                out_of_fold_predictions[holdout_index, i] = y_pred
                
        # Now train the cloned  meta-model using the out-of-fold predictions as new feature
        self.meta_model_.fit(out_of_fold_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        meta_features = np.column_stack([
            np.column_stack([model.predict(X) for model in base_models]).mean(axis=1)
            for base_models in self.base_models_ ])
        return self.meta_model_.predict(meta_features)

In [ ]:
start_time = time.time() 
stacked_averaged_models  = StackingAveragedModels(base_models = (lgb0, xgb0), meta_model = gbstf0)
stacked_averaged_models.fit(X_train, y_train)
y_pred = stacked_averaged_models.predict(X_val[train_cols])
score  = mean_absolute_precision_error(y_pred, y_val)
print("stacking Averaged models", '%.2f' % float((time.time() - start_time)/60 ) +" mins, score =", '%.2f' % score)
# stacking Averaged models 79.53 mins, score = 17.95

In [ ]:
class AveragingModels(BaseEstimator, RegressorMixin, TransformerMixin):
    def __init__(self, models):
        self.models = models
        
    # we define clones of the original models to fit the data in
    def fit(self, X, y):
        self.models_ = [clone(x) for x in self.models]
        
        # Train cloned base models
        for model in self.models_:
            model.fit(X, y)

        return self
    
    #Now we do the predictions for cloned models and average them
    def predict(self, X):
        predictions = np.column_stack([
            model.predict(X) for model in self.models_
        ])
        return np.mean(predictions, axis=1)

In [ ]:
start_time = time.time()
averaged_models = AveragingModels(models = (xgb0, lgb0))
averaged_models.fit(X_train, y_train)
train_cols = X_train.columns.tolist()
y_pred = averaged_models.predict(X_val[train_cols])
score  = mean_absolute_precision_error(y_pred, y_val)
print("Average models", '%.2f' % float((time.time() - start_time)/60 ) +" mins, score =", '%.2f' % score)
# Average models 30.67 mins, score=  17.01

# Final model

In [ ]:
# Current best model 
start_time = time.time()
GBST = GradientBoostingRegressor(n_estimators=3200, learning_rate=0.05,
                                  max_depth=10, max_features='sqrt',
                                  min_samples_leaf=15, min_samples_split=10, 
                                  loss='huber', random_state =5)

GBST.fit(train.drop(['ID','price'],axis=1), train.price)
y_pred_test = GBST.predict(test.drop(['ID','price'],axis=1))

print('%.2f' % float((time.time() - start_time)/60 ) +" mins.")

# Save to file

In [ ]:
filename = "/Users/mohsenkiskani/Downloads/Ubaar/submissions/submission32.csv"
with open(filename,"w+") as outputfile:
    outputfile.write("ID,price\n")
    for i in range(y_pred_test.shape[0]):
        outputfile.write(str(test_data.ID[i])+","+str(int(np.ceil(y_pred_test[i])))+"\n")